In [ ]:
import pytest
from pytest_bdd import scenario, given, when, then
import torch
from pyciemss.ODE.models import SIDARTHE
from pyciemss.utils import get_tspan
from pyciemss.ODE.askem_primitives import sample
import pyro.distributions as dist

# Unit Test 1

Set the initial values and parameters, as described in the Supplementary Methods section of the publication (pg. 9 of the pdf) and simulate for 100 days. Peak should be at day 47

evaluation_scenario_2.feature  file
```
Feature: Evaluation Scenario 2
    Reproduce the result in page 9 of the supplementary methods section of the SIDARTHE publication

    Scenario: Unit test 1
        Given initial conditions
        And parameters
        And SIDARTHE model

        When simulating the model for 100 days

        Then peak of infection is around day 47
```

In [ ]:
@scenario('evaluation_scenario_2.feature', 'Unit test 1')
def test_UnitTest1():
    pass


@given("initial conditions", target_fixture="initial_conditions")
def initial_conditions():
    num_samples = 100
    N = 1
    # Initial number of infected and recovered individuals, I0 and R0.
    I0, D0, A0, R0, T0, H0, E0 = 200/60e6, 20/60e6, 1/60e6, 2/60e6, 0, 0, 0
    # Everyone else
    S0 = N - I0 - D0 - A0 - R0 - T0 - H0 - E0
    return dict(N=1, initial_state=tuple(torch.as_tensor(s) for s in  (S0, I0, D0, A0, R0, T0, H0, E0)),
                final_observed_state = tuple(torch.as_tensor(s) for s in  (S0, I0, D0, A0, R0, T0, H0, E0)))

@given("parameters", target_fixture="parameters")
def parameters():
    return dict(alpha_prior=dist.Delta(torch.tensor (0.570)) ,
                beta_prior=dist.Delta(torch.tensor  (0.011)) ,
                gamma_prior=dist.Delta(torch.tensor (0.456)) ,
                delta_prior=dist.Delta(torch.tensor (0.011)) ,
                epsilon_prior=dist.Delta(torch.tensor(0.171)),
                lamb_prior =dist.Delta(torch.tensor (0.034)) ,
                zeta_prior=dist.Delta(torch.tensor  (0.125)) ,
                eta_prior=dist.Delta(torch.tensor   (0.125)) ,
                kappa_prior=dist.Delta(torch.tensor (0.017)) ,
                theta_prior=dist.Delta(torch.tensor (0.371)) ,
                rho_prior=dist.Delta(torch.tensor   (0.034)) ,
                xi_prior=dist.Delta(torch.tensor    (0.017)) ,
                sigma_prior=dist.Delta(torch.tensor (0.017)) ,
                mu_prior=dist.Delta(torch.tensor    (0.017)) ,
                nu_prior=dist.Delta(torch.tensor    (0.027)) ,
                tau_prior=dist.Delta(torch.tensor   (0.01))  )


@given("SIDARTHE model", target_fixture="initialize_SIDARTHE_model")
def initialize_SIDARTHE_model(initial_conditions, parameters):
    return SIDARTHE(initial_conditions["N"],**parameters)


@when("simulating the model for 100 days", target_fixture="simulate_for_days")
def simulate_for_days(initialize_SIDARTHE_model,initial_conditions,days=100):
    return sample(initialize_SIDARTHE_model, 1, initial_conditions["initial_state"], get_tspan(1, days, days))


@then("peak of infection is around day 47")
def day_around_47(simulate_for_days):
    assert torch.abs(torch.argmax(simulate_for_days["I_total_obs"]) - torch.tensor(47)) <= 3

# Unit Test 2

Now update the parameters to reflect various interventions that Italy implemented during the first wave, as described in detail on pg. 9. Simulate for 100 days, reproduce the trajectories in Fig. 2B, and determine the day and level of peak total infections (sum over all the infected states I, D, A, R, T). Expected output: Trajectories in Fig. 2B, peak occurs around day 50, with ~0.2% of the total population infected.

TLDR; Same test but using multiple new parameter sets. Peak at day 50 with y= ~0.2% (+or- 0.2)

In [ ]:
#evaluation_scenario_2.feature  file

#Feature: Evaluation Scenario 2
#    Reproduce the result in page 9 of the supplementary methods section of the SIDARTHE publication

    Scenario: Unit test 2
        Given initial conditions
        And parameters
        And SIDARTHE model

        When simulating the model for 100 days

        Then peak of infection is around day 47
        
Feature: Multiple site support

  Background:
    Given a global administrator named "Greg"
    And a blog named "Greg's anti-tax rants"
    And a customer named "Wilson"
    And a blog named "Expensive Therapy" owned by "Wilson"

  Scenario: Wilson posts to his own blog
    Given I am logged in as Wilson
    When I try to post to "Expensive Therapy"
    Then I should see "Your article was published."

  Scenario: Greg posts to a client's blog
    Given I am logged in as Greg
    When I try to post to "Expensive Therapy"
    Then I should see "Your article was published."